<a href="https://colab.research.google.com/github/AbouCamara/COURS-IA/blob/main/Classlocat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# This will import the python's module os
import os
from scipy import io
import cv2
import matplotlib.pyplot as plt
import pandas as pd
def extract_mat_contents(annot_directory, image_dir):
	        # Create MAT Parser
	        mat = scipy.io.loadmat(annot_directory)
	        # Get the height and width for our image
	        height, width = cv2.imread(image_dir).shape[:2]
	        # Get the bounding box co-ordinates
	        x1, y2, y1, x2 = tuple(map(tuple, mat['box_coord']))[0]
	        # We Split the image Directory passed in the method and choose the index
	        # Of the Folders name which is the same as it's class
	        class_name = image_dir.split('/')[2]
	        filename = '/'.join(image_dir.split('/')[-2:])
	        # Return the extracted attributes
	        return filename,  width, height, class_name, x1,y1,x2,y2

In [14]:
# Function to convert MAT files to CSV
def mat_to_csv(annot_directory, image_directory, classes_folders):

  # List containing all our attributes regarding each image
  mat_list = []

  # We loop our each class and its labels one by one to preprocess and augment 
  for class_folder in classes_folders:

    # Set our images and annotations directory
    image_dir = os.path.join(image_directory, class_folder)
    annot_dir = os.path.join(annot_directory, class_folder) 

    # Get each file in the image and annotation directory
    mat_files = sorted(os.listdir(annot_dir))
    img_files = sorted(os.listdir(image_dir))

    # Loop over each of the image and its label
    for mat, image_file in zip(mat_files, img_files):
      
      # Full mat path
      mat_path = os.path.join(annot_dir, mat)

      # Full path Image
      img_path = os.path.join(image_dir, image_file)

      # Get Attributes for each image 
      value = extract_mat_contents(mat_path, img_path)

      # Append the attributes to the mat_list
      mat_list.append(value)

  # Columns for Pandas DataFrame
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 
                 'xmax', 'ymax']

  # Create the DataFrame from mat_list
  mat_df = pd.DataFrame(mat_list, columns=column_name)

  # Return the dataframe
  return mat_df

# The Classes we will use for our training
classes_list = sorted(['butterfly',  'cougar_face', 'elephant'])


# Set our images and annotations directory
image_directory = 'CALTECH/CALTECH_Dataset'
annot_directory = 'CALTECH/CALTECH_Annotations'

# Run the function to convert all the MAT files to a Pandas DataFrame
labels_df = mat_to_csv(annot_directory, image_directory, classes_list)

# Saving the Pandas DataFrame as CSV File
labels_df.to_csv(('labels.csv'), index=None)